# Problema 2

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=11c373f65a3013c767de187a5573149d1b498067c646100e4051346297fe06f2
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark import SparkContext

# Create the Spark context
sc = SparkContext("local", "ShortestPath")

In [8]:
# Definir la función de actualización del costo acumulado en los nodos
def update_node_cost(node, cost):
    if cost < node[1]:
        return (node[0], cost)
    else:
        return node

In [10]:
# Nodo inicial indicado por el usuario
initial_node = 1

# RDD de aristas
#edges = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)]
edges = [(1, 2, 5), (1, 4, 20), (2, 3, 10), (2, 4, 25), (4, 3, 10)]

# Crear el RDD de aristas
edges_rdd = sc.parallelize(edges)

# Inicializar los nodos con costo acumulado infinito, excepto el nodo inicial
nodes_rdd = edges_rdd.flatMap(lambda edge: [edge[0], edge[1]]).distinct().map(lambda node: (node, float('inf')))
nodes_rdd = nodes_rdd.map(lambda node: (node[0], 0) if node[0] == initial_node else node)

change = 0
while True:
  neighbors_rdd = edges_rdd.map(lambda edge: (edge[0], (edge[1], edge[2])))
  joined_rdd = nodes_rdd.join(neighbors_rdd)

  updated_rdd = joined_rdd.map(lambda x: (x[1][1][0], x[1][0] + x[1][1][1]))
  nodes_rdd = nodes_rdd.union(updated_rdd).reduceByKey(lambda x, y: min(x, y)).map(lambda node: update_node_cost(node, node[1]))

  changes = nodes_rdd.filter(lambda node: node[1] == float('inf')).count()

  # Salir del bucle si no hay cambios en los costos de los nodos
  if changes == change:
      break
  else:
    change = changes


result = nodes_rdd.collect()

# Imprimir el resultado
for node in result:
    print(f"Nodo: {node[0]}, Costo: {node[1]}")

Nodo: 1, Costo: 0
Nodo: 2, Costo: 5
Nodo: 3, Costo: 15
Nodo: 4, Costo: 20
